# Poly model
Working on a version of `design_linear.py` that takes a numpy  Polynomial class (or a list of coefficients) instead of a single value to apply linaerly as m * x + 0.0

In [11]:
import numpy as np
from numpy.polynomial import Polynomial

import cmm_error_map.design_poly as designp
import cmm_error_map.design_linear as designl
import cmm_error_map.data_cmpts as dc
import cmm_error_map.config.config as cf

In [12]:
model_params = dc.model_parameters_test.copy()


In [13]:
coeffs = {
    key: value if isinstance(value, list) else [0.0, value]
    for key, value in model_params.items()
}
polys = {key: Polynomial(coeffs[key]) for key in model_params}

In [14]:
xyz_in = np.array([10.0, 20.0, 30.0])
xyzt = np.array([0.0, 0.0, -250.0])

In [15]:
designp.poly_dependency(xyz_in, polys)

{'Txx': 0.000133,
 'Txy': 0.0,
 'Txz': 0.0,
 'Tyx': -0.000224,
 'Tyy': -0.00010180000000000001,
 'Tyz': 0.0,
 'Tzx': 0.00078,
 'Tzy': 0.000138,
 'Tzz': 1.002e-06,
 'Rxx': 7.489999999999999e-08,
 'Rxy': 1.5399999999999998e-07,
 'Rxz': 5e-08,
 'Ryx': -9.16e-08,
 'Ryy': -2.86e-07,
 'Ryz': 4.3800000000000003e-07,
 'Rzx': 7.47e-08,
 'Rzy': -2.382e-08,
 'Rzz': 1.434e-06,
 'Wxy': 0.0,
 'Wxz': 0.0,
 'Wyz': 0.0}

In [16]:
designp.model_point(xyz_in, xyzt, polys)

array([7.22994912e-04, 5.17014183e-05, 9.60117688e-07])

In [18]:
designl.linear_model_point(xyz_in, xyzt, model_params)

array([7.22994912e-04, 5.17014183e-05, 9.60117688e-07])

## profiling
`design_poly.poly_dependency`

In [23]:
%load_ext line_profiler

In [20]:
ballspacing = 133.0
x0, y0, z0 = 250.0, 50.0, 50.0
# XY plane
transform_mat_xy = dc.matrix_from_vectors([x0, y0, z0], [0.0, 0.0, 0.0])
xt, yt, zt = 0.0, 0.0, -243.4852
prb_xy = dc.Probe(title="P0", name="p0", length=np.array([xt, yt, zt]))
mmt_xy = dc.Measurement(
    title="Plate XY",
    name="mmt_00",
    artefact=cf.artefact_models["KOBA 0620"],
    transform_mat=transform_mat_xy,
    probe=prb_xy,
    cmm_nominal=None,
    cmm_dev=None,
    mmt_nominal=None,
    mmt_dev=None,
)
mmt_xy.recalculate(model_params, dc.pmm_866.cmm_model)

In [21]:
%timeit designp.model_matrix(mmt_xy.cmm_nominal.T, mmt_xy.probe.length, model)

4.36 ms ± 154 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%lprun -f  designp.model_point designp.model_matrix(mmt_xy.cmm_nominal.T, mmt_xy.probe.length, model)

Timer unit: 1e-09 s

Total time: 0.0389952 s
File: /home/elfnor/gits/ls_cmm_error_map/src/cmm_error_map/design_poly.py
Function: model_point at line 81

Line #      Hits         Time  Per Hit   % Time  Line Contents
    81                                           def model_point(
    82                                               xyz_in: np.ndarray,  # (3,)
    83                                               xyzt: np.ndarray,  # (3,)
    84                                               model_polys: dict[str, Polynomial],
    85                                               fixed_table=False,
    86                                               bridge_axis=1,
    87                                           ):
    88                                               """ """
    89        25      57381.0   2295.2      0.1      if not fixed_table:
    90                                                   # table movement is in opposite direction to probe position for a moving table
    91 